# Synapse Labeling Orphans

In [1]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [2]:
dj.config['display.limit'] = 20
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.SynapseExclude()

In [4]:
schema = dj.schema('microns_ta3p100')

@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> ta3p100.Synapse
    ---
    (postsynaptic_label)->ta3p100.LabelKey
    """ # (postsynaptic_label)->ta3p100.LabelKey : tinyint unsigned

In [5]:
segment_ids = (ta3p100.CoarseLabelOrphan - ta3p100.SegmentExclude).fetch('segment_id')
mesh_fetch = (ta3p100.CleansedMeshOrphan & ta3p100.CompartmentOrphan.proj()).fetch()
label_fetch = (ta3p100.CoarseLabelOrphan - ta3p100.SegmentExclude).fetch()

meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6])
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7])
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((ta3p100.Synapse & ta3p100.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

100%|██████████| 572/572 [03:13<00:00,  3.15it/s]


In [6]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ Old way of inserting without length threshold
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]"""
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=2,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/572 [00:00<?, ?it/s]
0it [00:00, ?it/s]
164it [00:00, 107428.68it/s]
0it [00:00, ?it/s]
  0%|          | 2/572 [00:00<00:57,  9.86it/s]
0it [00:00, ?it/s]
37it [00:00, 30145.54it/s]
0it [00:00, ?it/s]
  1%|          | 4/572 [00:00<00:52, 10.84it/s]
0it [00:00, ?it/s]
  1%|          | 5/572 [00:00<00:59,  9.54it/s]
0it [00:00, ?it/s]
  1%|          | 6/572 [00:00<01:05,  8.66it/s]
0it [00:00, ?it/s]
11it [00:00, 14049.13it/s]
0it [00:00, ?it/s]
  1%|▏         | 8/572 [00:00<00:54, 10.33it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
73it [00:00, 78188.00it/s]
0it [00:00, ?it/s]
  2%|▏         | 11/572 [00:01<00:55, 10.14it/s]
0it [00:00, ?it/s]
82it [00:00, 88391.91it/s]
0it [00:00, ?it/s]
19it [00:00, 27603.66it/s]
0it [00:00, ?it/s]
  2%|▏         | 14/572 [00:01<01:22,  6.78it/s]
0it [00:00, ?it/s]
45it [00:00, 38876.14it/s]
0it [00:00, ?it/s]
  3%|▎         | 16/572 [00:02<01:25,  6.52it/s]
0it [00:00, ?it/s]
  3%|▎         | 17/572 [00:02<01:22,  6.74it/s]
0it [00:00

In [18]:
print(len(synapses_to_exclude))

1996


In [19]:
ta3p100.SynapseExcludeFinal.insert(synapses_to_exclude,skip_duplicates=True)

In [7]:
segmentation = ta3p100.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 572/572 [00:00<00:00, 4765.97it/s]


17.767342805862427


# Spine Synapse Labeler

In [8]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> ta3p100.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->ta3p100.LabelKey
    """ # (spine_label)->ta3p100.LabelKey : tinyint unsigned

In [9]:
mesh_fetch = np.array((ta3p100.CleansedMeshOrphan & ta3p100.CompartmentOrphan.proj()).fetch('segment_id', 'vertices', 'triangles')).T
label_fetch = np.array(ta3p100.OverlayedSpineLabelOrphanRevised.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((ta3p100.Synapse & ta3p100.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 572/572 [03:51<00:00,  2.39it/s]


In [10]:
matched_synapses = dict()
synapses_to_exclude_spine = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    """ OLD WAY WITHOUTH LENGTH THRESHOLD
    for synapse_id, nearest_node in zip(synapse_ids, nearest_nodes):
        synapse_labels[synapse_id] = vertex_labels[nearest_node]
    """
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude_spine.append(dict(segmentation=2,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/572 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/572 [00:00<00:58,  9.68it/s]
0it [00:00, ?it/s]
158it [00:00, 120908.60it/s]
0it [00:00, ?it/s]
  1%|          | 3/572 [00:00<00:51, 10.95it/s]
0it [00:00, ?it/s]
  1%|          | 4/572 [00:00<00:55, 10.18it/s]
0it [00:00, ?it/s]
  1%|          | 5/572 [00:00<00:58,  9.76it/s]
0it [00:00, ?it/s]
  1%|          | 6/572 [00:00<01:05,  8.65it/s]
0it [00:00, ?it/s]
11it [00:00, 12192.74it/s]
0it [00:00, ?it/s]
  1%|▏         | 8/572 [00:00<00:54, 10.29it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
73it [00:00, 70177.44it/s]
0it [00:00, ?it/s]
  2%|▏         | 11/572 [00:01<00:54, 10.28it/s]
0it [00:00, ?it/s]
82it [00:00, 69579.80it/s]
0it [00:00, ?it/s]
19it [00:00, 20460.02it/s]
0it [00:00, ?it/s]
  2%|▏         | 14/572 [00:01<01:18,  7.10it/s]
0it [00:00, ?it/s]
45it [00:00, 76662.75it/s]
0it [00:00, ?it/s]
  3%|▎         | 16/572 [00:02<01:20,  6.92it/s]
0it [00:00, ?it/s]
  3%|▎         | 17/572 [00:02<01:17, 

In [11]:
#####STILL NEED TO WRITE THE SYNAPSES TO EXCLUDE TO DATABASE

In [12]:
print(len(synapses_to_exclude_spine))


1996


In [17]:
#add to synapse exclude
ta3p100.SynapseExcludeFinal.insert(synapses_to_exclude_spine,skip_duplicates=True)

In [13]:
segmentation = ta3p100.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 572/572 [00:00<00:00, 4163.07it/s]


24.226929903030396


## Visualization of Labeled Synapses (no labeled Spines yet)

In [14]:
# ta3p100.LabelKey()

In [15]:
# ta3p100.ProofreadLabel & 'segment_id=648518346341393609'

In [16]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'ignore blue'), ( 3, 'Basal', 'ignore yellow'),
#                ( 4, 'Oblique', 'ignore green'), ( 5, 'Soma', 'ignore red'),
#                ( 6, 'Axon-Soma', 'ignore aqua'), ( 7, 'Axon-Dendr', 'ignore off blue'),
#                ( 8, 'Dendrite', 'ignore purple'), ( 9, 'Distal', 'ignore pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'ignore tan'),
#                (12, 'Cilia', 'ignore orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'ignore green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()